In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

pergunta = input("Pergunta: ")

# Prompt explícito com exemplos
prompt = f"""<|system|>Você é um assistente útil que responde sempre em português de forma clara e objetiva.<|end|>
<|user|>Qual é a capital da França?<|end|>
<|assistant|>A capital da França é Paris.<|end|>
<|user|>Quem descobriu o Brasil?<|end|>
<|assistant|>O Brasil foi descoberto por Pedro Álvares Cabral em 1500.<|end|>
<|user|>{pergunta}<|end|>
<|assistant|>"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

saida = tokenizer.decode(outputs[0], skip_special_tokens=True)
resposta = saida.split("<|assistant|>")[-1].strip()
print("Resposta:", resposta)


Pergunta: qual é a capital dos estados unidos?
Resposta: A capital dos Estados Unidos é Washington, D.C.<|end|>
